In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tf
import os

In [ ]:
zstack_path = '/Volumes/data_jm_share/data_raw/calibration/z_psf imaging-001/zseries imaging-001_Cycle00001_Ch1_000001.ome.tif'
zstack = tf.imread(zstack_path)


In [ ]:
x_artefact_relative_to_argmax = np.array([0, 2, 4, 6, 8, 10, 12, 14])
y_artefact = np.array([1700, 1800, 2100, 3400, 5000, 4000, 2500, 1500])

In [ ]:
zstack.shape

z_psf = np.mean(zstack, axis=(1,2))

# fit gaussian
from scipy.optimize import curve_fit

def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))
x = np.arange(len(z_psf))
p0 = [np.max(z_psf), np.argmax(z_psf), 1]
popt, pcov = curve_fit(gaussian, x, z_psf, p0=p0)
# get width at half max

half_max = popt[0] / 2
half_max_index = np.where(z_psf >= half_max)[0]
half_max_index = half_max_index[0]
half_max_width = np.abs(half_max_index - np.where(z_psf >= half_max)[0][-1])
print('img_z_psf FWHM:', half_max_width)
z_psf_argmax = np.argmax(z_psf)

x_artefact = x_artefact_relative_to_argmax + z_psf_argmax

artefact_argmax = np.argmax(y_artefact)
artefact_argmax = x_artefact[artefact_argmax]


plt.figure(figsize=(6, 3), dpi=300)

plt.plot(z_psf)


plt.plot(x, z_psf, 'C0', label='imaging')
plt.plot(x, gaussian(x, *popt), 'grey', alpha=0.5, label='sigma=%5.3f' % (popt[2]))
plt.plot(x_artefact, y_artefact, 'C1', label='artefact')


plt.axvline(z_psf_argmax, color='C0', alpha=0.5, linestyle='--', label='peak imaging')
plt.axvline(artefact_argmax, color='C1', alpha=0.5, linestyle='--', label='peak artefact')

plt.title(f'img_z_psf FWHM: {half_max_width} um\n peak_offset: {artefact_argmax - z_psf_argmax} um')

plt.xlim(15, 55)

plt.xlabel('z position (um)')

plt.legend(loc = 'upper left', fontsize=8)
plt.show()

In [ ]:
print(pcov)